In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from numpy import arange
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [3]:
benign_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_1400.csv')
benign_df.head(2)

,average_packet_size,bwd_IAT_total,bwd_PSH_flags,bwd_URG_flags,bwd_header_length,bwd_packets_per_second,bwd_segment_size_avg,bwd_win_bytes,download_upload_ratio,flow_bytes_per_second,...,idlePacket_std,packet_lenght_max,packet_lenght_mean,packet_lenght_min,packet_lenght_std,totalPacketFeature_backward,totalPacketFeature_forward,totalPacketFeature_length_of_backward,totalPacketFeature_length_of_forward,Label
0,0.122062,1.825124,0.0,0.0,-0.089724,-0.093257,0.235996,-0.152944,-0.424199,-0.152383,...,1.177924,0.602600,0.122820,-0.176185,0.639749,-0.088560,-0.099721,-0.065056,-0.086993,1
1,-0.225433,-0.677532,0.0,0.0,-0.093663,0.052214,-0.252308,-0.190380,2.150248,-0.090449,...,-0.877381,-0.345148,-0.273383,0.782710,-0.417382,-0.092324,-0.107229,-0.066490,-0.087427,1


In [5]:
portmap_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\Reindex\Portmap.csv')
portmap_df

,Unnamed: 0,average_packet_size,bwd_IAT_total,bwd_PSH_flags,bwd_URG_flags,bwd_header_length,bwd_packets_per_second,bwd_segment_size_avg,bwd_win_bytes,download_upload_ratio,...,idlePacket_min,idlePacket_std,packet_lenght_max,packet_lenght_mean,packet_lenght_min,packet_lenght_std,totalPacketFeature_backward,totalPacketFeature_forward,totalPacketFeature_length_of_backward,totalPacketFeature_length_of_forward
0,0,0.00,0.0,0,0,0,0.000000,0.0,-1,0.0,...,9048097.0,351582.631269,0.0,0.000000,0.0,0.000000,0,45,0.0,0.0
1,1,0.00,0.0,0,0,0,0.000000,0.0,-1,0.0,...,8820294.0,351541.079539,0.0,0.000000,0.0,0.000000,0,56,0.0,0.0
2,2,31.75,2.0,0,0,40,0.054892,46.0,250,0.0,...,36373056.0,0.000000,46.0,28.222222,6.0,21.081851,2,6,92.0,116.0
3,3,31.75,2.0,0,0,40,0.054893,46.0,252,0.0,...,36372291.0,0.000000,46.0,28.222222,6.0,21.081851,2,6,92.0,116.0
4,4,31.75,2.0,0,0,40,0.054893,46.0,246,0.0,...,36372216.0,0.000000,46.0,28.222222,6.0,21.081851,2,6,92.0,116.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191689,191689,343.50,0.0,0,0,0,0.000000,0.0,-1,0.0,...,0.0,0.000000,229.0,229.000000,229.0,0.000000,0,2,0.0,458.0
191690,191690,343.50,0.0,0,0,0,0.000000,0.0,-1,0.0,...,0.0,0.000000,229.0,229.000000,229.0,0.000000,0,2,0.0,458.0
191691,191691,343.50,0.0,0,0,0,0.000000,0.0,-1,0.0,...,0.0,0.000000,229.0,229.000000,229.0,0.000000,0,2,0.0,458.0
191692,191692,343.50,0.0,0,0,0,0.000000,0.0,-1,0.0,...,0.0,0.000000,229.0,229.000000,229.0,0.000000,0,2,0.0,458.0


In [6]:
portmap_df.drop("Unnamed: 0", axis=1, inplace=True)

In [7]:
portmap_df['Label'] = -1

In [26]:

portmap_df.isnull().sum()
portmap_df.replace([np.inf, -np.inf], np.nan, inplace=True)
portmap_df.dropna(inplace=True)

In [21]:
benign_test_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_600.csv')

In [27]:
# novelty detection
df_test = pd.concat([benign_test_df, portmap_df], ignore_index=True)
df_train = benign_df

# training_data
X_train = df_train.drop(['Label'], axis=1)
y_train = df_train['Label']
# testing_data
X_test = df_test.drop(['Label'], axis=1)
y_test = df_test['Label']

In [7]:
# df_test.to_csv(r'D:\Kuliah\ProyekAkhir\Data Weka\benign_portmap.csv', index = False, header=True)

In [30]:
# Novelty detection with 8000 sample balanced
for a in range(1, 20):
    for i in arange(0.1, 0.5, 0.1):
        clf = LocalOutlierFactor(n_neighbors=a, novelty=True, contamination=i)
        clf.fit(X_train)
        y_pred_test = clf.predict(X_test)
        print('Neighbour : ' + str(a))
        print('contamination : ' + str(i))
        print('---Classification Report---')
        print(classification_report(y_test, y_pred_test))

Neighbour : 1
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.92      0.96       634

    accuracy                           1.00    182528
   macro avg       1.00      0.96      0.98    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 1
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.81      0.90       634

    accuracy                           1.00    182528
   macro avg       1.00      0.91      0.95    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 1
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.73  

Neighbour : 6
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.80      0.89       634

    accuracy                           1.00    182528
   macro avg       1.00      0.90      0.94    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 6
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.68      0.81       634

    accuracy                           1.00    182528
   macro avg       1.00      0.84      0.91    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 6
contamination : 0.4
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.56  

Neighbour : 11
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.73      0.84       634

    accuracy                           1.00    182528
   macro avg       1.00      0.86      0.92    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 11
contamination : 0.4
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.59      0.74       634

    accuracy                           1.00    182528
   macro avg       1.00      0.79      0.87    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 12
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.9

Neighbour : 16
contamination : 0.4
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.60      0.75       634

    accuracy                           1.00    182528
   macro avg       1.00      0.80      0.88    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 17
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.88      0.94       634

    accuracy                           1.00    182528
   macro avg       1.00      0.94      0.97    182528
weighted avg       1.00      1.00      1.00    182528

Neighbour : 17
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00    181894
           1       1.00      0.79      0.88     